# Get FAQ
Data Crawling & Preprocessing

### 1. Import libraries

In [1]:
import requests
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup

### 2. FAQ for KT AIVLE trainees
Cookie values ​​are required for crawling. My SESSION value has been masked, so it needs to be modified to use this code.

In [2]:
headers = {'User-Agnet': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',}
url = 'https://aivle.edu.kt.co.kr/home/brd/faq/list'
cookie = {'SESSION': 'your_session_value'}

response = requests.post(url, headers=headers, cookies=cookie)
response


<Response [200]>

In [3]:
# Extract FAQ's questions and answers from html file
soup = BeautifulSoup(response.text, 'html.parser')

faq_list = soup.find_all('li', class_='tr')
faq_type = '진행'
results = []

for faq in faq_list:
    question = faq.find('button', class_='question').text.strip()
    answer = faq.find('div', class_='ask').text.strip()
    if answer != '':
        result = {'구분': faq_type, 'QA': f"{question.strip()}\n{answer.strip()}"}
        results.append(result)

faq_df = pd.DataFrame(results)
faq_df.head()

,구분,QA
0,진행,자리 이석 시 어떻게 해야하나요?\n이석으로 자리를 비울 경우 외출을 사용해 주시면...
1,진행,출석인정요청은 어떻게 하나요?\n출결에 대한 출석인정요청은 D(사유발생일)+7일 이...
2,진행,"휴가 제도는 어떻게 되나요?\n휴가는 단위기간당 1회 사용 가능하며, 교육은 정상 ..."
3,진행,학교 수업 및 시험은 출결 인정이 되나요?\n학과수업 및 시험의 경우 출석 인정이 ...
4,진행,국민취업제도 상담을 위한 외출도 출결에 반영되나요?\n국취제 상담은 공가처리 불가하...


In [4]:
faq_df.to_csv('faq_for_aivler.csv')

### 3. FAQ for AIVLE school applicants
Cookie values ​​are required for crawling. My SESSION value has been masked, so it needs to be modified to use this code.

In [5]:
# Check server connection availability & check total page count

headers = {'User-Agnet': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',}
cookie = {'SESSION': '9b077d54-cd88-4b7d-9d26-e61e7f427ed4'}

url = f'https://aivle.kt.co.kr/home/brd/faq/listJson'
response = requests.get(url, headers=headers, cookies=cookie)
totalPageCount = response.json()['pageInfo']['totalPageCount']

response, totalPageCount


(<Response [200]>, 7)

In [6]:
results = []

for i in range(1, totalPageCount+1):
    url = f'https://aivle.kt.co.kr/home/brd/faq/listJson?&pageIndex={i}'
    response_json = requests.get(url, headers=headers, cookies=cookie).json()
    
    for data in response_json['returnList']:
        question = data['atclTitle']        # 질문
        answer = data['atclCts']            # 답변
        faq_type = data['ctgrNm']           # 구분
        if answer != '':
            # sp: 질문의 시작점(start poing), 질문의 인덱싱을 제거하기 위함 -> 예: '1)'
            sp = question.find(')')+1
            # 질문과 답변은 하나의 chunk로 구성
            result = {'구분': faq_type, 'QA': f"{question[sp:].strip()}\n{answer.strip()}"}
            results.append(result)
    

faq_df2 = pd.DataFrame(results)
faq_df2.sample(3)

,구분,QA
48,국민내일배움카드,훈련장려금이 무엇인가요?\n<p>훈련장려금이란 긴 기간의 국비과정을 수강하는 분들에...
19,모집/선발,(지원서 작성) 현재 4학년 1학기 재학 또는 휴학 등의 사유로 졸업(예정)증명서 ...
17,모집/선발,인적성검사는 KT 채용과 같은 과목인가요?\n현재 KT 에이블스쿨은 KT 채용 인적...


In [7]:
faq_df2.to_csv('faq_for_candidate.csv')